In [44]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import numpy as np

stopwords = set(stopwords.words("english"))

def listtostring(clean_line):
    str1 = " "
    return(str1.join(clean_line))

#using Lemmatization
lemma = nltk.wordnet.WordNetLemmatizer()

file_docs = []
test_doc = []
test_line = []

with open('E:\JS\Py\Text_similarity\Idea_list.csv', 'r') as inp, open('E:\JS\Py\Text_similarity\Idea_List.txt', 'w') as out:
    for line in inp:
        out.write(line + '.\n')

with open ('E:\JS\Py\Text_similarity\Idea_List.txt') as f:
    tokens = sent_tokenize(f.read())
    #wtokens = word_tokenize(f.read())
    for line in tokens:
        file_docs.append(line)
        

# create a clean document after removing the stop-words and generate lemmas
clean_doc = []
for line in file_docs:
    clean_line = [word for word in line.split() if word not in stopwords]
    
    for word in line.split():
        test_line.append(lemma.lemmatize(word))
    test_doc.append(listtostring(test_line))
    test_line = []
    
    #clean_doc.append(listtostring(clean_line))
    clean_doc = test_doc

print("Number of documents:",len(clean_doc))

# gen_docs = [[w.lower() for w in word_tokenize(text)] 
#             for text in clean_doc]

gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in clean_doc]

dictionary = gensim.corpora.Dictionary(gen_docs)
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

tf_idf = gensim.models.TfidfModel(corpus)

# for doc in tf_idf[corpus]:
#     print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

# building the index
sims = gensim.similarities.Similarity('E:/JS/Py/Text_similarity/',tf_idf[corpus],
                                        num_features=len(dictionary))

file2_docs = []
#test_doc = []
#test_line = []

with open('E:\JS\Py\Text_similarity\Idea_query.csv', 'r') as inp, open('E:\JS\Py\Text_similarity\Idea_query.txt', 'w') as out:
    for line in inp:
        out.write(line + '.\n')

with open ('E:\JS\Py\Text_similarity\Idea_query.txt') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file2_docs.append(line)
        
test_doc = []
clean_doc2 = []
for line in file2_docs:
    clean_line = [word for word in line.split() if word not in stopwords]
    
    for word in line.split():
        test_line.append(lemma.lemmatize(word))
    test_doc.append(listtostring(test_line))
    test_line = []
    
    #clean_doc.append(listtostring(clean_line))
    clean_doc2 = test_doc

print("Number of documents:",len(clean_doc2))  
for line in clean_doc2:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc) #update an existing dictionary and create bag of words
        
    # building the index
    sims = gensim.similarities.Similarity('E:/JS/Py/Text_similarity/',tf_idf[corpus],
                                            num_features=len(dictionary))

    # perform a similarity query against the corpus
    query_doc_tf_idf = tf_idf[query_doc_bow]

    #print("\n",clean_doc2,"\n")

    filter_result = filter(lambda x: x[1]>0.5, enumerate(sims[query_doc_tf_idf]))
    final_result = sorted(filter_result, key=lambda x: x[1], reverse=True)
  
    #print(final_result)
    #print matching entries with similarity scores
    for s in final_result:
        print(s[0], round(s[1]*100,2), ": " + file_docs[s[0]])

#print('\nFiltered and Sorted Result:\n', sorted(filter_result, key=lambda x: x[1], reverse=True))

Number of documents: 6
Number of documents: 2
0 87.33 : LHD control mounting reinforcement deletion from RHD BIW
.
5 99.16 : Sunroof cover mechanism deletion
.


In [73]:
import nltk
from pywsd.lesk import simple_lesk
import numpy as np
from nltk.corpus import wordnet as wn

class SentenceSimilarity:
    
    def __init__(self):
        self.word_order = False
        
    
    def identifyWordsForComparison(self, sentence):
        #Taking out Noun and Verb for comparison word based
        tokens = nltk.word_tokenize(sentence)        
        pos = nltk.pos_tag(tokens)
        pos = [p for p in pos if p[1].startswith('N') or p[1].startswith('V') or p[1].startswith('R')]     
        return pos
    
    def wordSenseDisambiguation(self, sentence):
        # removing the disambiguity by getting the context
        pos = self.identifyWordsForComparison(sentence)
        sense = []
        for p in pos:
            sense.append(simple_lesk(sentence, p[0], pos=p[1][0].lower()))
        return set(sense)
    
#     def getSimilarity(self, arr1, arr2, vector_len):
#         #cross multilping all domains 
#         vector = [0.0] * vector_len
#         count = 0
#         for i,a1 in enumerate(arr1):
#             all_similarityIndex=[]
#             for a2 in arr2:
#                 similarity = wn.synset(a1.name()).wup_similarity(wn.synset(a2.name()))
#                 #similarity = a1.wup_similarity(a2)
#                 if similarity != None:
#                     all_similarityIndex.append(similarity)
#                 else:
#                     all_similarityIndex.append(0.0)
#             all_similarityIndex = sorted(all_similarityIndex, reverse = True)
#             vector[i]=all_similarityIndex[0]
#             if vector[i] >= 0.5:
#                 count +=1
#         return vector, count        
        
        
#     def shortestPathDistance(self, sense1, sense2):
#         #getting the shortest path to get the similarity
#         if len(sense1) >= len(sense2):
#             grt_Sense = len(sense1)
#             v1, c1 = self.getSimilarity(sense1, sense2, grt_Sense)
#             v2, c2 = self.getSimilarity(sense2, sense1, grt_Sense)
#         if len(sense2) > len(sense1):
#             grt_Sense = len(sense2)
#             v1, c1 = self.getSimilarity(sense2, sense1, grt_Sense)
#             v2, c2 = self.getSimilarity(sense1, sense2, grt_Sense)
#         return np.array(v1),np.array(v2),c1,c2
        
    def main(self, sentence1, sentence2):
        sense1 = self.wordSenseDisambiguation(sentence1)
        sense2 = self.wordSenseDisambiguation(sentence2)        
#         v1,v2,c1,c2 = self.shortestPathDistance(sense1,sense2)
#         dot = np.dot(v1,v2)
#         print("dot", dot) # getting the dot product
#         tow = (c1+c2)
#         final_similarity = dot/tow
#         print("similarity",final_similarity)
        print(sense1)
obj = SentenceSimilarity()
a = 'CCB weight reduction by using tube of lower thickness.'
b = 'SS.'
obj.main(a,b)

{Synset('thickness.n.03'), Synset('reduction.n.03'), Synset('use.v.06'), None, Synset('tube.n.01'), Synset('slant.v.02')}


In [69]:
sense1 = self.wordSenseDisambiguation('Sunroof mechanism cover deletion')

NameError: name 'self' is not defined